<a href="https://colab.research.google.com/github/nnugy-21/Coursera-2021/blob/main/04_Handling_Complex_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [40]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  # Your Code
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > DESIRED_ACCURACY):
      print("\nReached 100% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2021-03-24 16:38:01--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-03-24 16:38:01 (151 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [5]:
# Directory
train_happy_dir = os.path.join('/tmp/h-or-s/happy')

# Directory with our training human pictures
train_sad_dir = os.path.join('/tmp/h-or-s/sad')

In [6]:
train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-07.png', 'happy2-01.png', 'happy2-12.png', 'happy2-04.png', 'happy1-12.png', 'happy2-13.png', 'happy1-11.png', 'happy2-14.png', 'happy1-16.png', 'happy2-08.png']
['sad2-06.png', 'sad2-07.png', 'sad1-10.png', 'sad2-13.png', 'sad2-04.png', 'sad2-16.png', 'sad1-19.png', 'sad1-08.png', 'sad2-09.png', 'sad2-15.png']


In [8]:
print('total training happy images:', len(os.listdir(train_happy_dir)))
print('total training sad images:', len(os.listdir(train_sad_dir)))

total training happy images: 40
total training sad images: 40


In [44]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        # Your Code Here
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9248)             

In [45]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255) # Your Code Here Image

train_generator = train_datagen.flow_from_directory(
    # Your Code Here
    '/tmp/h-or-s/',  # This is the source directory for training images
    target_size=(150, 150),  # All images will be resized to 150x150
    batch_size=10,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [46]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit_generator(
    # Your Code Here
    train_generator,
    steps_per_epoch=2,  
    epochs=15,
    verbose=1,
    callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
2/2 [==============================] - 1s 260ms/step - loss: 1.7814 - accuracy: 0.6667
Epoch 2/15
2/2 [==============================] - 1s 252ms/step - loss: 1.0962 - accuracy: 0.4000
Epoch 3/15
2/2 [==============================] - 1s 240ms/step - loss: 0.6607 - accuracy: 0.7333
Epoch 4/15
2/2 [==============================] - 1s 253ms/step - loss: 1.1577 - accuracy: 0.4667
Epoch 5/15
2/2 [==============================] - 1s 255ms/step - loss: 0.5916 - accuracy: 0.5667
Epoch 6/15
2/2 [==============================] - 1s 258ms/step - loss: 0.4986 - accuracy: 0.6333
Epoch 7/15
2/2 [==============================] - 1s 254ms/step - loss: 0.4934 - accuracy: 0.8667
Epoch 8/15
2/2 [==============================] - 1s 262ms/step - loss: 0.4270 - accuracy: 0.7667
Epoch 9/15
2/2 [==============================] - 1s 247ms/step - loss: 0.5533 - accuracy: 0.6333
Epoch 10/15
2/2 [==============================] - 1s 253ms/step - loss: 0.1918 - accuracy: 1.0000

Reached 100% accur